In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3

/content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3


In [ ]:
!pip install beautifulsoup4 

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import pickle
import h5py
import csv
import string 

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text
def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [ ]:
train_df=pd.read_csv('task2_train_pmaug.csv')
val_df=pd.read_csv('task2_test.csv')

In [ ]:
train_df['text'] = train_df['text'].apply(text_preprocessing_pipeline)
val_df['text'] = val_df['text'].apply(text_preprocessing_pipeline)

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import torch.utils.data as data_utils

In [ ]:
class BasicModule(torch.nn.Module):
    def __init__(self):
        super(BasicModule, self).__init__()
        self.model_name = str(type(self))

    def load(self, path):
        self.load_state_dict(torch.load(path))

    def save(self, path=None):
        if path is None:
            raise ValueError('Please specify the saving road!!!')
        torch.save(self.state_dict(), path)
        return path


class StructuredSelfAttention(BasicModule):

    def __init__(self, batch_size, lstm_hid_dim, d_a, n_classes, label_embed, embeddings):
        super(StructuredSelfAttention, self).__init__()
        self.n_classes = n_classes
        self.embeddings = self._load_embeddings(embeddings)
        self.label_embed = self.load_labelembedd(label_embed)
        self.lstm = torch.nn.LSTM(300, hidden_size=lstm_hid_dim, num_layers=1,
                            batch_first=True, bidirectional=True)
        self.linear_first = torch.nn.Linear(lstm_hid_dim * 2, d_a) #W1 for attention
        self.linear_second = torch.nn.Linear(d_a, n_classes)       #W2 for attention

        self.weight1 = torch.nn.Linear(lstm_hid_dim * 2, 1)     #for adaptive fusion
        self.weight2 = torch.nn.Linear(lstm_hid_dim * 2, 1)     #for adaptive fusion

        self.output_layer = torch.nn.Linear(lstm_hid_dim*2, n_classes)
        self.embedding_dropout = torch.nn.Dropout(p=0.3)
        self.batch_size = batch_size
        self.lstm_hid_dim = lstm_hid_dim

    def _load_embeddings(self, embeddings):
        """Load the embeddings based on flag"""
        word_embeddings = torch.nn.Embedding(embeddings.size(0), embeddings.size(1))
        word_embeddings.weight = torch.nn.Parameter(embeddings)
        return word_embeddings
    def load_labelembedd(self, label_embed):
        """Load the embeddings based on flag"""
        embed = torch.nn.Embedding(label_embed.size(0), label_embed.size(1))
        embed.weight = torch.nn.Parameter(label_embed)
        return embed

    def init_hidden(self):
        return (torch.randn(2,self.batch_size,self.lstm_hid_dim).cuda(),torch.randn(2,self.batch_size,self.lstm_hid_dim).cuda())

    def forward(self,x):
        embeddings = self.embeddings(x)
        embeddings = self.embedding_dropout(embeddings)
        #step1 get LSTM outputs
        hidden_state = self.init_hidden()
        outputs, hidden_state = self.lstm(embeddings, hidden_state)
        print(outputs.size())
        #step2 get self-attention
        selfatt = torch.tanh(self.linear_first(outputs))
        print(selfatt.size())
        selfatt = self.linear_second(selfatt)
        print(selfatt.size())
        selfatt = F.softmax(selfatt, dim=1)
        print(selfatt.size())
        selfatt= selfatt.transpose(1, 2)
        print(selfatt.size())
        self_att = torch.bmm(selfatt, outputs)
        print(self_att.size())   
        #step3 get label-attention
        h1 = outputs[:, :, :self.lstm_hid_dim]
        h2 = outputs[:, :,self.lstm_hid_dim:]
        
        label = self.label_embed.weight.data
        m1 = torch.bmm(label.expand(self.batch_size, self.n_classes, self.lstm_hid_dim), h1.transpose(1, 2))
        m2 = torch.bmm(label.expand(self.batch_size, self.n_classes, self.lstm_hid_dim), h2.transpose(1, 2))
        label_att= torch.cat((torch.bmm(m1,h1),torch.bmm(m2,h2)),2)
        # label_att = F.normalize(label_att, p=2, dim=-1)
        # self_att = F.normalize(self_att, p=2, dim=-1) #all can
        weight1=torch.sigmoid(self.weight1(label_att))
        weight2 = torch.sigmoid(self.weight2(self_att ))
        weight1 = weight1/(weight1+weight2)
        weight2= 1-weight1

        doc = weight1*label_att+weight2*self_att
        # there two method, for simple, just add
        # also can use linear to do it
        avg_sentence_embeddings = torch.sum(doc, 1)/self.n_classes

        pred = torch.sigmoid(self.output_layer(avg_sentence_embeddings))
        return pred

In [ ]:
NUM_CALSSES=7

In [ ]:
def rec_label(tp, fn):
	if (tp + fn) == 0:
		return -np.inf
	return tp/float(tp + fn)

def prec_label(tp, fp):
	if (tp + fp) == 0:
		return -np.inf
	return tp/float(tp + fp)

def f_label(tp, fp, fn):
	if (2*tp + fn + fp) == 0:
		return -np.inf
	return (2*tp)/float(2*tp + fn + fp)

def components_F(pred, act):
	TP = np.zeros(7)
	FP = np.zeros(7)
	FN = np.zeros(7)
	for l_id in range(7):
		for pr, ac in zip(pred, act):
			if l_id in ac:
				if l_id in pr:
					TP[l_id] += 1
				else:
					FN[l_id] += 1
			else:
				if l_id in pr:
					FP[l_id] += 1
	return TP, FP, FN				

def F_metrics_label_macro(pred, act):
	TP, FP, FN = components_F(pred, act)
	return F_metrics_label_macro_from_comp(TP, FP, FN)

def F_metrics_label_macro_from_comp(TP, FP, FN):
	avgF = 0.0
	avgP = 0.0
	avgR = 0.0
	for tp, fp, fn in zip(TP,FP,FN):
		avgP += prec_label(tp, fp)
		avgR += rec_label(tp, fn)
		avgF += f_label(tp, fp, fn)

	return avgP/7, avgR/7, avgF/7

def F_metrics_label_micro(pred, act):
	TP, FP, FN = components_F(pred, act)
	return F_metrics_label_micro_from_comp(TP, FP, FN)

def F_metrics_label_micro_from_comp(TP, FP, FN):
	avgTP = np.mean(TP)
	avgFP = np.mean(FP)
	avgFN = np.mean(FN)
	return prec_label(avgTP, avgFP), rec_label(avgTP, avgFN), f_label(avgTP, avgFP, avgFN)
 
def f_indi(pred,act):
	TP, FP, FN = components_F(pred, act)
	pre_ind=[]
	rec_ind=[]
	f1_ind=[]
	for i in range(7):
		pre_ind.append(prec_label(TP[i],FP[i]))
		rec_ind.append(rec_label(TP[i],FN[i]))
		f1_ind.append(f_label(TP[i],FP[i],FN[i]))
	return pre_ind,rec_ind,f1_ind		

In [ ]:
from sklearn import metrics

In [ ]:
def train(attention_model,train_loader,test_loader,criterion,opt,epochs = 5,GPU=True):
    if GPU:
        attention_model.cuda()
    for i in range(epochs):
        print("Running EPOCH",i+1)
        train_loss = []
        #pre = []
        #recall = []
        #f1_micro=[]
        #f1_macro=[] 
        best_val = 0
        for batch_idx, train in enumerate(tqdm(train_loader)):
            opt.zero_grad()
            x, y = train[0].cuda(), train[1].cuda()
            y_pred= attention_model(x)
            loss = criterion(y_pred, y.float())/train_loader.batch_size
            loss.backward()
            opt.step()
            labels_cpu = y.data.cpu().float()
            pred_cpu = y_pred.data.cpu()
            #TP,FP,FN=components_F(pred_cpu.numpy(),labels_cpu.numpy())
            #pl_ma, rl_ma, fl_ma = F_metrics_label_macro_from_comp(TP, FP, FN)
            #_,_,fl_mi=F_metrics_label_micro_from_comp(TP, FP, FN)
            #pre.append(pl_ma)
            #recall.append(rl_ma)
            #f1_micro.append(fl_mi)
            #f1_macro.append(fl_ma)
            train_loss.append(float(loss))
        avg_loss = np.mean(train_loss)
        #epoch_prec = np.array(pre).mean(axis=0)
        #epoch_rec = np.array(recall).mean(axis=0)
        #epoch_f1mi = np.array(f1_micro).mean(axis=0)
        #epoch_f1ma = np.array(f1_macro).mean(axis=0)
        print("epoch %2d train end : avg_loss = %.4f" % (i+1, avg_loss))
        #print("precision : %.4f, recall : %.4f, f1_micro: %.4f, f1_macro: %.4f" % (epoch_prec,epoch_rec,epoch_f1mi,epoch_f1ma))
        test_loss = []
        outputs=[]
        targets=[]
        #test_pre = []
        #test_rec=[]
        #test_f1mi=[]
        #test_f1ma=[]
        #test_f1ind=[]
        for batch_idx, test in enumerate(tqdm(test_loader)):
            x, y = test[0].cuda(), test[1].cuda()
            val_y= attention_model(x)
            loss = criterion(val_y, y.float()) /train_loader.batch_size
            labels_cpu = y.data.cpu().detach().float()
            pred_cpu = val_y.data.cpu().detach()
            outputs.extend(pred_cpu.numpy().tolist())
            targets.extend(labels_cpu.numpy().tolist())
            #TP,FP,FN=components_F(pred_cpu.numpy(),labels_cpu.numpy())
            #pl_ma, rl_ma, fl_ma = F_metrics_label_macro_from_comp(TP, FP, FN)
            #_,_,fl_mi=F_metrics_label_micro_from_comp(TP, FP, FN)
            #test_pre.append(pl_ma)
            #test_rec.append(rl_ma)
            #test_f1mi.append(fl_mi)
            #test_f1ma.append(fl_ma)
            #est_f1ind.append(f_indi(pred_cpu.numpy(),labels_cpu.numpy()))
            test_loss.append(float(loss))
        avg_test_loss = np.mean(test_loss)
        outputs=np.array(outputs)>=0.5
        f1mi=metrics.f1_score(targets, outputs, average='micro')
        f1ma=metrics.f1_score(targets, outputs, average='macro')
        #test_pre = np.array(test_pre).mean(axis=0)
        #test_rec = np.array(test_rec).mean(axis=0)
        #test_f1mi = np.array(test_f1mi).mean(axis=0)
        #test_f1ma = np.array(test_f1ma).mean(axis=0)
        #test_f1ind=np.array(test_f1ind).mean(axis=0)
        print("epoch %2d test end : avg_loss = %.4f, f1_micro = %.4f, f1_macro= %.4f" % (i+1, avg_test_loss,f1mi,f1ma))
        #print("test_precision : %.4f, test_recall : %.4f, test_f1_micro: %.4f, test_f1_macro: %.4f" % (test_pre,test_rec,test_f1mi,test_f1ma))
        #print('test_prec_ind')
        #print(test_f1ind[0])
        #print('test_rec_ind')
        #print(test_f1ind[1])
        #print('test_f1_ind')
        #print(test_f1ind[2])
        '''if f1ma > best_val:
          state={'epoch': i+1, 'state_dict':attention_model.state_dict(),'optimizer': opt.state_dict()}
          torch.save(state,'./saved_models/LSAN_glove_bce_eda.pt')'''
        





In [ ]:
target_cols=['upr','ss','ps','av','m','c','pm']
X_train=train_df.text
X_test=val_df.text
y_train = torch.from_numpy(np.vstack(train_df[target_cols].values)).float()
y_test = torch.from_numpy(np.vstack(val_df[target_cols].values)).float()


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def loadWRVModel(File):
    print("Loading Word Representation Vector Model")
    f = open(File,'r')
    WRVModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        try:
          wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        except:
          print(splitLines[1:])
          print(len(splitLines[1:]))
          break
        WRVModel[word] = wordEmbedding
    print(len(WRVModel)," words loaded!")
    return WRVModel

In [ ]:
WRVModel = loadWRVModel('./glove/glove.6B.300d.txt')

Loading Word Representation Vector Model
400001  words loaded!


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

sequences_text_train = tokenizer.texts_to_sequences(X_train)
sequences_text_test = tokenizer.texts_to_sequences(X_test)

X_train = torch.from_numpy(pad_sequences(sequences_text_train, maxlen=128, padding='post'))
X_test = torch.from_numpy(pad_sequences(sequences_text_test, maxlen=128, padding='post'))

In [ ]:
print(X_train.shape)
print(X_test.shape)

torch.Size([1066, 128])
torch.Size([199, 128])


In [ ]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
embedding_matrix = torch.zeros(VOCAB_SIZE, 300)

unk = 0
for i in range(1, VOCAB_SIZE):
  word = tokenizer.index_word[i]
  if word in WRVModel.keys():
    embedding_matrix[i] = torch.from_numpy(WRVModel[word]).float()
  else:
    unk +=1
print('VOCAB_SIZE : {}'.format(VOCAB_SIZE))
print('TOTAL OF UNKNOWN WORD : {}'.format(unk))

VOCAB_SIZE : 6957
TOTAL OF UNKNOWN WORD : 229


In [ ]:
label_names=['unbalanced power relations','shallow solutions','presupposition','authority voice','metaphor','compassion','the poorer the merrier']
label_embedding = torch.zeros(7,300)

for index, label in enumerate(label_names):
  words = label.split()
  num_of_words = len(words)

  for sublabel in words:
    if sublabel in WRVModel.keys():
      label_embedding[index] +=  torch.from_numpy(WRVModel[sublabel])
  label_embedding[index] = label_embedding[index]/num_of_words

print(label_embedding)

tensor([[ 0.0918, -0.1889, -0.0252,  ..., -0.3856,  0.1331,  0.0991],
        [ 0.0658,  0.3624, -0.2718,  ...,  0.0158, -0.0659, -0.0381],
        [ 0.5054, -0.7354,  0.2486,  ..., -0.0017,  0.4011, -0.5620],
        ...,
        [-0.3232, -0.6007,  0.4608,  ...,  0.2982,  0.0444,  0.4491],
        [-0.5578, -0.4172, -0.4372,  ...,  0.4248,  0.8330, -0.3659],
        [-0.1084, -0.0585,  0.1369,  ..., -0.0645, -0.0293, -0.0310]])


In [ ]:
epochs=50
lstm_hidden_dimension=300
batch_size=64
d_a=200
emb_size=300
GPU=True
use_cuda=True
label_num=7

In [ ]:
train_data = data_utils.TensorDataset(X_train,y_train)
test_data = data_utils.TensorDataset(X_test,y_test)
train_loader = data_utils.DataLoader(train_data, batch_size, shuffle=True, drop_last=True)
test_loader = data_utils.DataLoader(test_data, batch_size, drop_last=True)

In [ ]:
def multilabel_classification(attention_model, train_loader, test_loader, epochs, GPU=True):
    loss = torch.nn.BCELoss()
    opt = torch.optim.Adam(attention_model.parameters(), lr=0.001, betas=(0.9, 0.99))
    train(attention_model, train_loader, test_loader, loss, opt, epochs,GPU)

attention_model = StructuredSelfAttention(batch_size=batch_size, lstm_hid_dim=lstm_hidden_dimension,
                                          d_a=d_a, n_classes=label_num, label_embed=label_embedding,embeddings=embedding_matrix)

In [ ]:
if use_cuda:
  attention_model.cuda()


In [ ]:
multilabel_classification(attention_model, train_loader, test_loader, epochs=epochs)

Running EPOCH 1


  0%|          | 0/16 [00:00<?, ?it/s]

torch.Size([64, 128, 600])
torch.Size([64, 128, 200])
torch.Size([64, 128, 7])
torch.Size([64, 128, 7])
torch.Size([64, 7, 128])
torch.Size([64, 7, 128])


  6%|▋         | 1/16 [00:00<00:03,  3.78it/s]

torch.Size([64, 128, 600])
torch.Size([64, 128, 200])
torch.Size([64, 128, 7])
torch.Size([64, 128, 7])
torch.Size([64, 7, 128])
torch.Size([64, 7, 128])


...

epoch  5 train end : avg_loss = 0.0053


  0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([64, 128, 600])
torch.Size([64, 128, 200])
torch.Size([64, 128, 7])
torch.Size([64, 128, 7])
torch.Size([64, 7, 128])
torch.Size([64, 7, 128])


100%|██████████| 3/3 [00:00<00:00, 38.03it/s]

torch.Size([64, 128, 600])
torch.Size([64, 128, 200])
torch.Size([64, 128, 7])
torch.Size([64, 128, 7])
torch.Size([64, 7, 128])
torch.Size([64, 7, 128])
torch.Size([64, 128, 600])
torch.Size([64, 128, 200])
torch.Size([64, 128, 7])
torch.Size([64, 128, 7])
torch.Size([64, 7, 128])
torch.Size([64, 7, 128])
epoch  5 test end : avg_loss = 0.0077, f1_micro = 0.6395, f1_macro= 0.4453
